In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Create imbalanced binary classification dataset
X, y = make_classification(n_samples=1000,
                           n_features=10,
                           n_informative=2,
                           n_redundant=8,
                           weights=[0.9, 0.1],
                           flip_y=0,
                           random_state=42
                           )

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    stratify=y,
                                                    random_state=42)

In [ ]:
# Define model hyperparameters

params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 42
}

lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

InvalidParameterError: The 'solver' parameter of LogisticRegression must be a str among {'liblinear', 'lbfgs', 'sag', 'newton-cg', 'saga', 'newton-cholesky'}. Got 'lgfbs' instead.